In [5]:
import pandas as pd
import numpy as np
import cufflinks as cf

ttm = 1
strike = 100
spot = 100
sigma = .2
rfr = .05
callput = "call"
nas = 20

ds = 2*strike/nas
dt = (0.9/sigma**2/nas**2) #time step size for stability

nts = (ttm/dt) + 1
dt = ttm/nts

s = np.arange(0,(nas+1)*ds,ds)
t = np.arange(0,ttm+dt,dt)


grid = np.zeros((len(s),len(t)))
grid = pd.DataFrame(grid, index = s, columns = np.around(t,3))

if callput == "call":
    grid.iloc[:,0] = np.maximum(s-strike,0)    #terminal condition for call
else:
    grid.iloc[:,0] = np.maximum(strike-s,0)    #terminal condition for put


for k in range (1,len(t)):
    for i in range (1,len(s)-1):
                delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
                gamma = (grid.iloc[i+1,k-1] - 2*grid.iloc[i,k-1] + grid.iloc[i-1,k-1])/(ds**2)
                theta = - delta*rfr*s[i] + rfr*grid.iloc[i,k-1] - .5*sigma**2*s[i]**2*gamma
                grid.iloc[i,k] = grid.iloc[i,k-1] - (theta*dt)
    
    grid.iloc[0,k] = grid.iloc[0,k-1] * (1-rfr*dt)       #boundary condition at spot 0.
    
    grid.iloc[len(s)-1,k] = 2*grid.iloc[len(s)-2,k] - grid.iloc[len(s)-3,k]      #boundary condition at spot tends to infinity.

grid = np.around(grid,3)

grid.iloc[:,:].iplot(kind = 'surface', title='Call Option values by Explicit FDM')

